## SV<sup>2</sup> Classifier Training

## Table of Contents

1. [Getting Started](#readme)

2. [Input](#input)
  * Get Your Training Data
     * [SV<sup>2</sup> Training Data](#sv2default)
     * [Generate New Training Data](#sv2train)
  * [Copy Number Input](#copynumber)

3. [Name Your Classifer](#clf)

4. Training
  * [Deletion > 1000 Classifier](#delgt)

  * [Deletion <= 1000 Classifier](#dellt)

  * [Deletion Male Sex Chromosomes Classifier](#delmsc)

  * [Duplication Breakpoint Classifier](#dupbrk)

  * [Duplication SNV Classifier](#dupsnv)

  * [Duplication Male Sex Chromosomes Classifier](#dupmsc)

5. [Loading New Classifiers](#dump)

In [1]:
%%html
<style>
table {margin-left: 0 !important;}
</style>

### This jupyter notebook is a guide for training the SVM genotyping classifers for SV<sup>2</sup>. <a class="anchor" id="readme"></a>

This notebook is currently formatted to train the default classifiers that SV<sup>2</sup> **versions >=1.2** uses.

The parameters for the classifier are the ones used for the default classifiers. To retrain the models, alter these values.

Feel free to edit this document to train your own data. 
  * Classifiers will be saved as pickle (.pkl) files. 
  * Paths to classifier pickle files are saved as a JSON file.

#### For questions please contact Danny Antaki: dantaki@ucsd.edu

## Input <a class="anchor" id="input"></a>

Get your training data

### 1. Train using the default training set <a class="anchor" id="sv2default"></a>

The default training set is located here

```
$ ls SV2-INSTALL-PATH/sv2/training/1kgp_training_data/*

    1kgp_highcov_del_gt1kb.txt  
    1kgp_highcov_del_lt1kb.txt  
    1kgp_highcov_del_malesexchrom.txt  
    1kgp_highcov_dup_snv.txt  
    1kgp_lowcov_dup_breakpoint.txt  
    1kgp_lowcov_dup_malesexchrom.txt

```

### 2. Generate training input with `sv2train` <a class="anchor" id="sv2train"></a>

```
sv2train -i <in.bam> -snv <snv.vcf.gz ...> -b <sv.bed ...> -v <sv.vcf ...> -p <in.ped ...> 
```

Training input files located here
```
$ ls sv2_training_features/*

    sv2_training_features_deletion_gt1kb.txt
    sv2_training_features_deletion_lt1kb.txt
    sv2_training_features_deletion_male_sex_chrom.txt
    sv2_training_features_duplication_breakpoint.txt
    sv2_training_features_duplication_male_sex_chrom.txt
    sv2_training_features_duplication_snv.txt

$ head -n 2 sv2_training_features/sv2_deletion_gt1kb_training_features.txt
```


| chr | start | end | type |	id  | covr | dpe_rat | sr_rat | ... | copy_number | classifier |
| :--- | ----- | --- | ---- |  --- | ---- | ------- | ------ | --- | ----------- | ---------- |
| chr1 | 193646553 | 193654283 | DEL  |	HG00096	| 1.145 | 0.0 |	0.0 | ... | **NA** |deletion_gt1kb |
 
This jupyter notebook will accept training data in a pandas DataFrame given the following column names

| column | description | 
| ------ | ----------- |
| covr   | depth of coverage  feature |
| dpe_rat | discordant paired-end feature |
| sr_rat  | split-read feature | 
| HET_ratio | heterozygous allele depth feature | 
| copy_number | [copy number](#copynumber) |

### Copy Number Input <a class="anchor" id="copynumber"></a> 

sv2train does not produce genotypes, hence copy_number values are not available (**NA**). The user will have to supply genotypes for training custom data sets. 

The included default SV<sup>2</sup> training sets contain copy_number values

This jupyter notebook expects the following copy_number values

#### Biallelic SVs

| copy_number | VCF genotype |
| :----------- | ------------ |
| 4           | 1/1 (HOM)    |
| 3           | 0/1 (HET)    | 
| 2           | 0/0  (REF)   |
| 1           | 0/1  (HET)   |
| 0           | 1/1  (HOM)   |

#### SVs on Male Sex Chromosomes

| copy_number | VCF genotype |
| :----------- | ------------ |
| 2           | 1  (ALT)   | 
| 1           | 0  (REF)   |
| 0           | 1  (ALT)   |



## YOUR_CLF <a class="anchor" id="clf"></a>

In [2]:
YOUR_CLF="default" # for custom classifiers please change this name

In [3]:
"""
         WARNING:
DO NOT ALTER THE KEY NAMES
"""
clf_json={YOUR_CLF : 
          {
              'delgt': None,   #deletion >1000bp clf
              'dellt' : None,  #deletion <= 1000bp clf
              'delmsc' : None, #deletion male sex chroms clf
              'dupbrk' : None, #duplication breakpoint clf
              'dupsnv' : None, #duplication snv clf
              'dupmsc' : None  #duplication male sex chroms clf
          }
         }

In [4]:
import json
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
from os import getcwd
import pandas as pd
from sklearn import svm
try: import cPickle as pickle
except ImportError: import pickle
pd.options.mode.chained_assignment = None  # default='warn'
%matplotlib inline

In [ ]:
# global functions
def EuclidDist(a,x,b,y,coef): return 1 / sqrt(( ((a-x)**2) + ((b-y)**2) + coef ) )
def openTrain(path): 
    df=pd.read_csv(path,sep="\t")
    return df[df['covr']< 5.0] # do not train on outliers
def dup_convert_msc(df):
    cn=[]
    for x in df['copy_number']:
        gt=1
        if x>1: gt=0
        cn.append(gt)
    df['copy_number']=cn
    return df
def dup_convert(df):
    # convert duplication copy numbers
    ## 3 -> 1 : HET
    ## 4 -> 0 : HOM
    cn=[]
    for x in df['copy_number']:
        gt=2
        if x==3: gt=1
        if x==4: gt=0
        cn.append(gt)
    df['copy_number']=cn
    return df
def biallelic_weight(df,gts,hetexp,homexp,coef=0.01):
        # generate weights for biallelic SVs
        ## genotypes (gts) must be ordered as [REF, HET, HOM]
        ## hetexp, homexp are expected HET and HOM normalized coverages
        ## coef to avoid dividing by zero
        final=pd.DataFrame()
        for gtype in gts:
                cn = df[df['copy_number'] == gtype]
                covr = cn['covr']
                cntr = 1.0
                if gtype == gts[1]: cntr = hetexp
                elif gtype == gts[2]: cntr = homexp
                weights = [ 1.0/(abs(cntr-x)+coef) for x in covr] #inverse distance weights
                cn['weights']=weights
                final = final.append(cn)
        return final
def allele_weight(df,gts,altexp,coef=0.01):
        # generate weights for SVs on male sex chroms
        ## genotypes (gts) must be ordered as [REF,ALT]
        ## altexp is the ALT expected normalized coverages
        ## coef to avoid dividing by zero
        final=pd.DataFrame()
        for gtype in gts:
                cn = df[df['copy_number'] == gtype]
                covr = cn['covr']
                cntr = 1.0
                if gtype == gts[1]: cntr = altexp
                weights = [ 1.0/(abs(cntr-x)+coef) for x in covr] #inverse distance weights
                cn['weights']=weights
                final = final.append(cn)
        return final
def SNV_weight(df,gts,hetexp,homexp,coef=0.01):
        # generate weights for SVs with heterozygous allele depth (HAD) features
        ## genotypes (gts) must be ordered as [REF,HET,HOM]
        ## hetexp, homexp are expected HET and HOM normalized coverages
        ## coef to avoid dividing by zero
        final=pd.DataFrame()
        cn2med = np.median(df[df['copy_number']==gts[0]]['HET_ratio']) # median HAD ratio for REF
        cn3med = np.median(df[df['copy_number']==gts[1]]['HET_ratio']) # median HAD ratio for HET
        cn4med = cn2med/2.0 # median HAD ratio for HOM 2.0 is a heuristic based on density estimates  
        for gtype in gts:
                wgts=[]
                cn = df[df['copy_number'] == gtype]
                covr = cn['covr']
                het = cn2med
                cntr = 1.0
                if gtype == gts[1]: het,cntr = cn3med,hetexp
                elif gtype == gts[2]: het,cntr=cn2med,homexp
                for ind,ele in cn.iterrows():
                        wgt = EuclidDist(cntr,ele['covr'],het,ele['HET_ratio'],coef) # inverse Euclidean distance weights
                        wgt2 = None
                        if gtype == gts[2]:
                                # HOM weight for AAAa
                                wgt2 = EuclidDist(cntr,ele['covr'],cn4med,ele['HET_ratio'],coef)
                        if wgt2 != None: 
                            # maximum weight for AAaa vs. AAAa
                            if wgt2 > wgt: wgt=wgt2
                        wgts.append(wgt)
                cn['weights']=wgts
                final = final.append(cn)
        return final
def transpose_features(df): return np.vstack(np.asarray( (df['covr'],df['dpe_rat'],df['sr_rat']), order = 'C', dtype='float' )).T
def transpose_snv(df): return np.vstack(np.asarray( (df['covr'],df['HET_ratio']), order = 'C', dtype='float' )).T

## Deletion > 1000bp Classifer <a class="anchor" id="delgt"></a>

In [ ]:
CLF_NAME = "1kgp_highcov_del_gt1kb_svm_clf.pkl"
df = openTrain("1kgp_training_data/1kgp_highcov_del_gt1kb.txt")
# generate weights
df = biallelic_weight(df,[2,1,0],0.5,0.0)
# prepare data structures
x,y,wgt = transpose_features(df), np.array(df['copy_number']), np.array(df['weights'])
# train the SVM
#### RBF KERNEL PARAMETERS ####
C = 0.01 
GAMMA = 10
###############################
MEM = 12000 # in MB
clf = svm.SVC(kernel='rbf', 
                     probability=True,
                     random_state=42,
                     C=C,
                     gamma=GAMMA,
                     class_weight='balanced',
                     cache_size=MEM)
# this might take awhile!
clf.fit(x,y,sample_weight=wgt)
# svm model persistance
pickle.dump(clf, open(CLF_NAME, 'wb'))
clf_json[YOUR_CLF]['delgt']=getcwd()+'/'+CLF_NAME

## Deletion <= 1000 Classifier <a class="anchor" id="dellt"></a>

In [ ]:
CLF_NAME = "1kgp_highcov_del_lt1kb_svm_clf.pkl"
df = openTrain("1kgp_training_data/1kgp_highcov_del_lt1kb.txt")
# generate weights
df = biallelic_weight(df,[2,1,0],0.5,0.0)
# prepare data structures
x,y,wgt = transpose_features(df), np.array(df['copy_number']), np.array(df['weights'])
# train the SVM
#### RBF KERNEL PARAMETERS ####
C = 1000
GAMMA = 1
###############################
MEM = 12000 # in MB
clf = svm.SVC(kernel='rbf', 
                     probability=True,
                     random_state=42,
                     C=C,
                     gamma=GAMMA,
                     class_weight='balanced',
                     cache_size=MEM)
# this might take awhile!
clf.fit(x,y,sample_weight=wgt)
# svm model persistance
pickle.dump(clf, open(CLF_NAME, 'wb'))
clf_json[YOUR_CLF]['dellt']=getcwd()+'/'+CLF_NAME

## Deletion Male Sex Chrom Classifer <a class="anchor" id="delmsc"></a>

In [ ]:
CLF_NAME = "1kgp_highcov_del_malesexchrom_svm_clf.pkl"
df = openTrain("1kgp_training_data/1kgp_highcov_del_malesexchrom.txt")
# generate weights
df = allele_weight(df,[1,0],0.0)
# prepare data structures
x,y,wgt = transpose_features(df), np.array(df['copy_number']), np.array(df['weights'])
# train the SVM
#### RBF KERNEL PARAMETERS ####
C = 1
GAMMA = 1
###############################
MEM = 12000 # in MB
clf = svm.SVC(kernel='rbf', 
                     probability=True,
                     random_state=42,
                     C=C,
                     gamma=GAMMA,
                     class_weight='balanced',
                     cache_size=MEM)
# this might take awhile!
clf.fit(x,y,sample_weight=wgt)
# svm model persistance
pickle.dump(clf, open(CLF_NAME, 'wb'))
clf_json[YOUR_CLF]['delmsc']=getcwd()+'/'+CLF_NAME

## Duplication Breakpoint Classifier <a class="anchor" id="dupbrk"></a>

In [ ]:
CLF_NAME = "1kgp_lowcov_dup_breakpoint_svm_clf.pkl"
df = dup_convert(openTrain("1kgp_training_data/1kgp_lowcov_dup_breakpoint.txt"))
# generate weights
df = biallelic_weight(df,[2,1,0],1.5,2.0)
# prepare data structures
x,y,wgt = transpose_features(df), np.array(df['copy_number']), np.array(df['weights'])
# define class weights NOTE: you may want to try balanced weights before doing this!
class_wgt = { 
    0 : 2,
    1 : 0.5,
    2 : 1
}
# train the SVM
#### RBF KERNEL PARAMETERS ####
C = 1
GAMMA = 10
###############################
MEM = 12000 # in MB
clf = svm.SVC(kernel='rbf', 
                     probability=True,
                     random_state=42,
                     C=C,
                     gamma=GAMMA,
                     class_weight=class_wgt, # custom class weights
                     cache_size=MEM)
# this might take awhile!
clf.fit(x,y,sample_weight=wgt)
# svm model persistance
pickle.dump(clf, open(CLF_NAME, 'wb'))
clf_json[YOUR_CLF]['dupbrk']=getcwd()+'/'+CLF_NAME

## Duplication SNV Classifier <a class="anchor" id="dupsnv"></a>

In [ ]:
CLF_NAME = "1kgp_highcov_dup_snv_svm_clf.pkl"
df = dup_convert(openTrain("1kgp_training_data/1kgp_highcov_dup_snv.txt"))
# define some types (just in case)
df[['HET_ratio']]=df[['HET_ratio']].astype(float)
# generate weights
df = SNV_weight(df,[2,1,0],1.5,2.0)
# prepare data structures
x,y,wgt = transpose_snv(df), np.array(df['copy_number']), np.array(df['weights'])
# train the SVM
#### RBF KERNEL PARAMETERS ####
C = 0.01
GAMMA = 5000
###############################
MEM = 12000 # in MB
clf = svm.SVC(kernel='rbf', 
                     probability=True,
                     random_state=42,
                     C=C,
                     gamma=GAMMA,
                     class_weight='balanced',
                     cache_size=MEM)
# this might take awhile!
clf.fit(x,y,sample_weight=wgt)
# svm model persistance
pickle.dump(clf, open(CLF_NAME, 'wb'))
clf_json[YOUR_CLF]['dupsnv']=getcwd()+'/'+CLF_NAME

## Duplication Male Sex Chrom Classifer <a class="anchor" id="dupmsc"></a>

In [ ]:
CLF_NAME = "1kgp_lowcov_dup_malesexchrom_svm_clf.pkl"
df = dup_convert_msc(openTrain("1kgp_training_data/1kgp_lowcov_dup_malesexchrom.txt")) 
# generate weights
df = allele_weight(df,[1,0],2.0) # expected ALT in the default training set. You may want to change this for custom training
# prepare data structures
x,y,wgt = transpose_features(df), np.array(df['copy_number']), np.array(df['weights'])
# define class weights NOTE: you may want to try balanced weights before doing this!
class_wgt = { 
    0 : 1,
    1 : 1
}
# train the SVM
#### RBF KERNEL PARAMETERS ####
C = 1000
GAMMA = 0.01
###############################
MEM = 12000 # in MB
clf = svm.SVC(kernel='rbf', 
                     probability=True,
                     random_state=42,
                     C=C,
                     gamma=GAMMA,
                     class_weight=class_wgt, # custom class weights
                     cache_size=MEM)
# this might take awhile!
clf.fit(x,y,sample_weight=wgt)
# svm model persistance
pickle.dump(clf, open(CLF_NAME, 'wb'))
clf_json[YOUR_CLF]['dupmsc']=getcwd()+'/'+CLF_NAME

## Loading Classifiers to SV<sup>2</sup> <a class="anchor" id="dump"></a>

## **!** Run the cell below to save your work **!**

In [ ]:
outjson = YOUR_CLF+".json"
ofh = open(outjson,'w')
json.dump(clf_json,ofh,indent=4)
ofh.close()

### Run the following commands below
```
# add classifiers to SV2
sv2 -load-clf <YOUR_CLF.json>

# genotype with custom classifiers
sv2 ... -clf <YOUR_CLF>

# for example:
$ sv2 -load-clf default.json
$ sv2 -clf default
```

### Contact: Danny Antakli dantaki@ucsd.edu